In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_start = pd.read_csv('Data Sample (AIC) для итоговой работы (new).csv', na_values = ' ',decimal = ',')
df_start['Quantity'].astype(int)
df_start

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/19 8:26,3.39,17850.0,United Kingdom
1,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,12/1/19 8:34,1.69,13047.0,United Kingdom
2,536367,21755,LOVE BUILDING BLOCK WORD,3,12/1/19 8:34,5.95,13047.0,United Kingdom
3,536368,22914,BLUE COAT RACK PARIS FASHION,3,12/1/19 8:34,4.95,13047.0,United Kingdom
4,536373,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/19 9:02,2.75,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
54209,581585,84832,ZINC WILLIE WINKIE CANDLE STICK,24,12/9/20 12:31,0.85,15804.0,United Kingdom
54210,581585,23084,RABBIT NIGHT LIGHT,12,12/9/20 12:31,2.08,15804.0,United Kingdom
54211,581585,84692,BOX OF 24 COCKTAIL PARASOLS,25,12/9/20 12:31,0.42,15804.0,United Kingdom
54212,581587,22631,CIRCUS PARADE LUNCH BOX,12,12/9/20 12:50,1.95,12680.0,France


In [ ]:
df_start.describe()

,Quantity,UnitPrice,CustomerID
count,54214.000000,54214.000000,40643.000000
mean,9.610802,4.307341,15283.422803
std,57.358422,47.834496,1707.982278
min,-9600.000000,0.000000,12347.000000
25%,1.000000,1.250000,13969.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16775.000000
max,2400.000000,6930.000000,18287.000000


In [ ]:
unknown = df_start['Quantity'].count() - df_start['CustomerID'].count()
unknown

13571

In [ ]:
print(df_start[df_start['UnitPrice'] == 0.0].count()[5]) #цена 6й столбец, следовательно берем 5й индекс

225


In [ ]:
print(df_start.isna().mean().round(4) * 100)

InvoiceNo       0.00
StockCode       0.00
Description     0.25
Quantity        0.00
InvoiceDate     0.00
UnitPrice       0.00
CustomerID     25.03
Country         0.00
dtype: float64


На этом этапе видно, что данные мы частично потряли. Общее количество строк 54214, а заполненных строк с  CustomerID 40643. У 13571 транзакции не указан CustomerID, что составляет 25,03% всех транзакций.
в 0,25% транзакций не указано наименование товара.  
В 225 транзациях в датасете цена 0,0.
Предложения по датасету: проставить цены и наименования на основании имеющихся кодов товаров.
Не вполне понятно что делать с пропусками CustomerID
Варианты:
1. Вести расчеты не учитывая эти строки (В этом случае потеряем деньги или наоборот неучтем расходы.)
2. Исключить CustomerID из анализа (Теряем метрики)
3. Рассчитывать пользовательские метрики с поправкой на то, что у нас данные неполные. (наиболее привлекательный на мой взгляд вариант)

Пока что у меня ощущение, что часть данных с сайта и на нем есть авторизация, а часть из физического магазина и в магазине не предусмотрен сбор этих данных. Либо сбой ПО.



Теперь посмотрим на количество товара у которого не указано наименование.
Мои предложения по датасету:
1. Собрать на основе заполненных строк, так называемый ассортимент. Исходя из того, что наименование товара написано прописными буквами.
2. Отобрать уникальные штрих-коды
3. На основании этого ассортимента сопоставить наименования товара и цены.

Что бы я предложила для заказчику на этом этапе:
1. Мы убираем все неполные строки как то: строки без наименования товара, нулевые цены, всех пользователей без ID таким образом из всего датасета останется 40639 строк.
2. Проставляем цены и наименования на основании ассортимента. Тех пользователей у которых не указан ID учитываем только по среднему чеку. Отрицательные значения в графе количество я понимаю как расходы (бой,возврат, порча и т.д.)

В дальнейших рассчетах я исходила из 2 варианта.



In [ ]:
#пример датасета из варианта 1
df_start_pure = df_start.copy()
df_start_pure = df_start_pure[df_start_pure['UnitPrice'] > 0].dropna()
df_start_pure

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/19 8:26,3.39,17850.0,United Kingdom
1,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,12/1/19 8:34,1.69,13047.0,United Kingdom
2,536367,21755,LOVE BUILDING BLOCK WORD,3,12/1/19 8:34,5.95,13047.0,United Kingdom
3,536368,22914,BLUE COAT RACK PARIS FASHION,3,12/1/19 8:34,4.95,13047.0,United Kingdom
4,536373,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/19 9:02,2.75,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
54209,581585,84832,ZINC WILLIE WINKIE CANDLE STICK,24,12/9/20 12:31,0.85,15804.0,United Kingdom
54210,581585,23084,RABBIT NIGHT LIGHT,12,12/9/20 12:31,2.08,15804.0,United Kingdom
54211,581585,84692,BOX OF 24 COCKTAIL PARASOLS,25,12/9/20 12:31,0.42,15804.0,United Kingdom
54212,581587,22631,CIRCUS PARADE LUNCH BOX,12,12/9/20 12:50,1.95,12680.0,France


In [ ]:
test = df_start[df_start['Description'].isna()==True]
test

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
204,536553,37461,NaN,3,12/1/19 14:35,0.0,NaN,United Kingdom
730,536995,35951,NaN,57,12/3/19 15:29,0.0,NaN,United Kingdom
731,536998,85067,NaN,-6,12/3/19 15:30,0.0,NaN,United Kingdom
732,537005,21692,NaN,-70,12/3/19 15:35,0.0,NaN,United Kingdom
733,537009,84534B,NaN,-80,12/3/19 15:38,0.0,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
48602,577608,35400,NaN,5,11/21/20 9:22,0.0,NaN,United Kingdom
49770,578360,84971L,NaN,2,11/24/20 10:36,0.0,NaN,United Kingdom
50021,578693,21811,NaN,1,11/25/20 9:38,0.0,NaN,United Kingdom
52171,580359,20775,NaN,-35,12/2/20 16:11,0.0,NaN,United Kingdom


In [ ]:
df_product_range = df_start.copy()
# на основании датасета выводим ассортимент магазина:
df_product_range = df_product_range.iloc[:,[1,2,5]]
# выбираем всё, что написано прописными буквами
df_product_range = df_product_range.loc[df_product_range.Description.str.islower() == False]
# убираем дубликаты
df_product_range = df_product_range.drop_duplicates('StockCode')
df_product_range

,StockCode,Description,UnitPrice
0,84029E,RED WOOLLY HOTTIE WHITE HEART.,3.39
1,84879,ASSORTED COLOUR BIRD ORNAMENT,1.69
2,21755,LOVE BUILDING BLOCK WORD,5.95
3,22914,BLUE COAT RACK PARIS FASHION,4.95
4,84406B,CREAM CUPID HEARTS COAT HANGER,2.75
...,...,...,...
53447,21420,OYSTER TRINKET BOX,0.79
53917,90210C,RED ACRYLIC FACETED BANGLE,1.00
54049,23616,SET 10 CARDS JINGLE BELLS 17217,2.49
54088,90181B,AMETHYST GLASS/SHELL/PEARL NECKLACE,9.15


In [ ]:
df3 = pd.merge(df_start,df_product_range, on = 'StockCode', how = 'left')
df3

,InvoiceNo,StockCode,Description_x,Quantity,InvoiceDate,UnitPrice_x,CustomerID,Country,Description_y,UnitPrice_y
0,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/19 8:26,3.39,17850.0,United Kingdom,RED WOOLLY HOTTIE WHITE HEART.,3.39
1,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,12/1/19 8:34,1.69,13047.0,United Kingdom,ASSORTED COLOUR BIRD ORNAMENT,1.69
2,536367,21755,LOVE BUILDING BLOCK WORD,3,12/1/19 8:34,5.95,13047.0,United Kingdom,LOVE BUILDING BLOCK WORD,5.95
3,536368,22914,BLUE COAT RACK PARIS FASHION,3,12/1/19 8:34,4.95,13047.0,United Kingdom,BLUE COAT RACK PARIS FASHION,4.95
4,536373,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/19 9:02,2.75,17850.0,United Kingdom,CREAM CUPID HEARTS COAT HANGER,2.75
...,...,...,...,...,...,...,...,...,...,...
54209,581585,84832,ZINC WILLIE WINKIE CANDLE STICK,24,12/9/20 12:31,0.85,15804.0,United Kingdom,ZINC WILLIE WINKIE CANDLE STICK,0.85
54210,581585,23084,RABBIT NIGHT LIGHT,12,12/9/20 12:31,2.08,15804.0,United Kingdom,RABBIT NIGHT LIGHT,2.08
54211,581585,84692,BOX OF 24 COCKTAIL PARASOLS,25,12/9/20 12:31,0.42,15804.0,United Kingdom,BOX OF 24 COCKTAIL PARASOLS,0.42
54212,581587,22631,CIRCUS PARADE LUNCH BOX,12,12/9/20 12:50,1.95,12680.0,France,CIRCUS PARADE LUNCH BOX,3.36


In [ ]:
df3['Description'] = df3.apply(lambda x: x['Description_y'] if not pd.isnull(x['Description_y']) else x['Description_x'], axis = 1)
df3

,InvoiceNo,StockCode,Description_x,Quantity,InvoiceDate,UnitPrice_x,CustomerID,Country,Description_y,UnitPrice_y,Description
0,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/19 8:26,3.39,17850.0,United Kingdom,RED WOOLLY HOTTIE WHITE HEART.,3.39,RED WOOLLY HOTTIE WHITE HEART.
1,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,12/1/19 8:34,1.69,13047.0,United Kingdom,ASSORTED COLOUR BIRD ORNAMENT,1.69,ASSORTED COLOUR BIRD ORNAMENT
2,536367,21755,LOVE BUILDING BLOCK WORD,3,12/1/19 8:34,5.95,13047.0,United Kingdom,LOVE BUILDING BLOCK WORD,5.95,LOVE BUILDING BLOCK WORD
3,536368,22914,BLUE COAT RACK PARIS FASHION,3,12/1/19 8:34,4.95,13047.0,United Kingdom,BLUE COAT RACK PARIS FASHION,4.95,BLUE COAT RACK PARIS FASHION
4,536373,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/19 9:02,2.75,17850.0,United Kingdom,CREAM CUPID HEARTS COAT HANGER,2.75,CREAM CUPID HEARTS COAT HANGER
...,...,...,...,...,...,...,...,...,...,...,...
54209,581585,84832,ZINC WILLIE WINKIE CANDLE STICK,24,12/9/20 12:31,0.85,15804.0,United Kingdom,ZINC WILLIE WINKIE CANDLE STICK,0.85,ZINC WILLIE WINKIE CANDLE STICK
54210,581585,23084,RABBIT NIGHT LIGHT,12,12/9/20 12:31,2.08,15804.0,United Kingdom,RABBIT NIGHT LIGHT,2.08,RABBIT NIGHT LIGHT
54211,581585,84692,BOX OF 24 COCKTAIL PARASOLS,25,12/9/20 12:31,0.42,15804.0,United Kingdom,BOX OF 24 COCKTAIL PARASOLS,0.42,BOX OF 24 COCKTAIL PARASOLS
54212,581587,22631,CIRCUS PARADE LUNCH BOX,12,12/9/20 12:50,1.95,12680.0,France,CIRCUS PARADE LUNCH BOX,3.36,CIRCUS PARADE LUNCH BOX


Чем дольше смотришь на данные, тем больше вопросов они вызывают. Вот некоторые из вопросов, которые мне хотелось бы уточнить у заказчика, для более полного понимания:

1.  Если есть штрих код, то как может быть разная цена за единицу у товаров с одним штрихом? (дисконт? уценка?)
2.  Что обозначают транзакции ниже?
3.  Откуда в цене берутся 0? Как это происходит?
4.  Каким образом формируется датасет? (Это выгрузка из программы или это ведется в ручную.Есть ли возможность модернизировать датасет)
5. Что значат отрицательные значения в количестве?



In [ ]:
df3_odds = df3[df3['Description_x'].str.islower() == True]
df3_odds


,InvoiceNo,StockCode,Description_x,Quantity,InvoiceDate,UnitPrice_x,CustomerID,Country,Description_y,UnitPrice_y,Description
5538,540978,84050,counted,-310,1/12/20 15:04,0.0,NaN,United Kingdom,PINK HEART SHAPE EGG FRYING PAN,4.21,PINK HEART SHAPE EGG FRYING PAN
11573,546018,85172,wrongly sold as sets,-600,3/8/20 17:23,0.0,NaN,United Kingdom,HYACINTH BULB T-LIGHT CANDLES,0.42,HYACINTH BULB T-LIGHT CANDLES
11921,546407,22719,wrong barcode (22467),-178,3/11/20 16:24,0.0,NaN,United Kingdom,GUMBALL MONOCHROME COAT RACK,2.51,GUMBALL MONOCHROME COAT RACK
13309,547559,72759,thrown away-can't sell.,-524,3/23/20 17:27,0.0,NaN,United Kingdom,NaN,NaN,thrown away-can't sell.
13442,547704,21621,rcvd be air temp fix for dotcom sit,100,3/24/20 17:51,0.0,NaN,United Kingdom,VINTAGE UNION JACK BUNTING,8.50,VINTAGE UNION JACK BUNTING
14779,548999,84743C,damages,-26,4/5/20 14:34,0.0,NaN,United Kingdom,NaN,NaN,damages
16932,551019,22467,wrong code?,-100,4/26/20 11:19,0.0,NaN,United Kingdom,GUMBALL COAT RACK,2.55,GUMBALL COAT RACK
17882,552187,23209,mailout,100,5/6/20 13:59,0.0,NaN,United Kingdom,LUNCH BAG DOILEY PATTERN,1.65,LUNCH BAG DOILEY PATTERN
19336,553394,15058A,wet/rusty,-30,5/16/20 16:47,0.0,NaN,United Kingdom,BLUE POLKADOT GARDEN PARASOL,10.79,BLUE POLKADOT GARDEN PARASOL
20136,554143,79030G,damages,-413,5/23/20 10:40,0.0,NaN,United Kingdom,"TUMBLER, NEW ENGLAND",12.72,"TUMBLER, NEW ENGLAND"


In [ ]:
df3 = df3.drop(['Description_y','Description_x'],axis = 1)
df3


,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice_x,CustomerID,Country,UnitPrice_y,Description
0,536365,84029E,6,12/1/19 8:26,3.39,17850.0,United Kingdom,3.39,RED WOOLLY HOTTIE WHITE HEART.
1,536367,84879,32,12/1/19 8:34,1.69,13047.0,United Kingdom,1.69,ASSORTED COLOUR BIRD ORNAMENT
2,536367,21755,3,12/1/19 8:34,5.95,13047.0,United Kingdom,5.95,LOVE BUILDING BLOCK WORD
3,536368,22914,3,12/1/19 8:34,4.95,13047.0,United Kingdom,4.95,BLUE COAT RACK PARIS FASHION
4,536373,84406B,8,12/1/19 9:02,2.75,17850.0,United Kingdom,2.75,CREAM CUPID HEARTS COAT HANGER
...,...,...,...,...,...,...,...,...,...
54209,581585,84832,24,12/9/20 12:31,0.85,15804.0,United Kingdom,0.85,ZINC WILLIE WINKIE CANDLE STICK
54210,581585,23084,12,12/9/20 12:31,2.08,15804.0,United Kingdom,2.08,RABBIT NIGHT LIGHT
54211,581585,84692,25,12/9/20 12:31,0.42,15804.0,United Kingdom,0.42,BOX OF 24 COCKTAIL PARASOLS
54212,581587,22631,12,12/9/20 12:50,1.95,12680.0,France,3.36,CIRCUS PARADE LUNCH BOX


In [ ]:
df3['UnitPrice'] = df3.apply(lambda x: x['UnitPrice_y'] if x['UnitPrice_x'] == 0.0 else x['UnitPrice_x'], axis = 1)
df3

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice_x,CustomerID,Country,UnitPrice_y,Description,UnitPrice
0,536365,84029E,6,12/1/19 8:26,3.39,17850.0,United Kingdom,3.39,RED WOOLLY HOTTIE WHITE HEART.,3.39
1,536367,84879,32,12/1/19 8:34,1.69,13047.0,United Kingdom,1.69,ASSORTED COLOUR BIRD ORNAMENT,1.69
2,536367,21755,3,12/1/19 8:34,5.95,13047.0,United Kingdom,5.95,LOVE BUILDING BLOCK WORD,5.95
3,536368,22914,3,12/1/19 8:34,4.95,13047.0,United Kingdom,4.95,BLUE COAT RACK PARIS FASHION,4.95
4,536373,84406B,8,12/1/19 9:02,2.75,17850.0,United Kingdom,2.75,CREAM CUPID HEARTS COAT HANGER,2.75
...,...,...,...,...,...,...,...,...,...,...
54209,581585,84832,24,12/9/20 12:31,0.85,15804.0,United Kingdom,0.85,ZINC WILLIE WINKIE CANDLE STICK,0.85
54210,581585,23084,12,12/9/20 12:31,2.08,15804.0,United Kingdom,2.08,RABBIT NIGHT LIGHT,2.08
54211,581585,84692,25,12/9/20 12:31,0.42,15804.0,United Kingdom,0.42,BOX OF 24 COCKTAIL PARASOLS,0.42
54212,581587,22631,12,12/9/20 12:50,1.95,12680.0,France,3.36,CIRCUS PARADE LUNCH BOX,1.95


In [ ]:
df3 = df3.drop(['UnitPrice_x','UnitPrice_y'], axis = 1)
df3

,InvoiceNo,StockCode,Quantity,InvoiceDate,CustomerID,Country,Description,UnitPrice
0,536365,84029E,6,12/1/19 8:26,17850.0,United Kingdom,RED WOOLLY HOTTIE WHITE HEART.,3.39
1,536367,84879,32,12/1/19 8:34,13047.0,United Kingdom,ASSORTED COLOUR BIRD ORNAMENT,1.69
2,536367,21755,3,12/1/19 8:34,13047.0,United Kingdom,LOVE BUILDING BLOCK WORD,5.95
3,536368,22914,3,12/1/19 8:34,13047.0,United Kingdom,BLUE COAT RACK PARIS FASHION,4.95
4,536373,84406B,8,12/1/19 9:02,17850.0,United Kingdom,CREAM CUPID HEARTS COAT HANGER,2.75
...,...,...,...,...,...,...,...,...
54209,581585,84832,24,12/9/20 12:31,15804.0,United Kingdom,ZINC WILLIE WINKIE CANDLE STICK,0.85
54210,581585,23084,12,12/9/20 12:31,15804.0,United Kingdom,RABBIT NIGHT LIGHT,2.08
54211,581585,84692,25,12/9/20 12:31,15804.0,United Kingdom,BOX OF 24 COCKTAIL PARASOLS,0.42
54212,581587,22631,12,12/9/20 12:50,12680.0,France,CIRCUS PARADE LUNCH BOX,1.95


In [ ]:
#заполняем 0,0 то что не смогли найти
df3['UnitPrice'] = df3['UnitPrice'].fillna(0.0)

In [ ]:
#добавляем столбец сумма:
df3['Costs'] = round(df3.apply(lambda x: x['Quantity'] * x['UnitPrice'] if x['UnitPrice']> 0 else 0.0, axis = 1), 4)
df3

,InvoiceNo,StockCode,Quantity,InvoiceDate,CustomerID,Country,Description,UnitPrice,Costs
0,536365,84029E,6,12/1/19 8:26,17850.0,United Kingdom,RED WOOLLY HOTTIE WHITE HEART.,3.39,20.34
1,536367,84879,32,12/1/19 8:34,13047.0,United Kingdom,ASSORTED COLOUR BIRD ORNAMENT,1.69,54.08
2,536367,21755,3,12/1/19 8:34,13047.0,United Kingdom,LOVE BUILDING BLOCK WORD,5.95,17.85
3,536368,22914,3,12/1/19 8:34,13047.0,United Kingdom,BLUE COAT RACK PARIS FASHION,4.95,14.85
4,536373,84406B,8,12/1/19 9:02,17850.0,United Kingdom,CREAM CUPID HEARTS COAT HANGER,2.75,22.00
...,...,...,...,...,...,...,...,...,...
54209,581585,84832,24,12/9/20 12:31,15804.0,United Kingdom,ZINC WILLIE WINKIE CANDLE STICK,0.85,20.40
54210,581585,23084,12,12/9/20 12:31,15804.0,United Kingdom,RABBIT NIGHT LIGHT,2.08,24.96
54211,581585,84692,25,12/9/20 12:31,15804.0,United Kingdom,BOX OF 24 COCKTAIL PARASOLS,0.42,10.50
54212,581587,22631,12,12/9/20 12:50,12680.0,France,CIRCUS PARADE LUNCH BOX,1.95,23.40


In [ ]:
#подготовленный датасет из которого убираем те наименования, что не нашли
df_prepared = df3.dropna(subset='Description')
df_prepared

,InvoiceNo,StockCode,Quantity,InvoiceDate,CustomerID,Country,Description,UnitPrice,Costs
0,536365,84029E,6,12/1/19 8:26,17850.0,United Kingdom,RED WOOLLY HOTTIE WHITE HEART.,3.39,20.34
1,536367,84879,32,12/1/19 8:34,13047.0,United Kingdom,ASSORTED COLOUR BIRD ORNAMENT,1.69,54.08
2,536367,21755,3,12/1/19 8:34,13047.0,United Kingdom,LOVE BUILDING BLOCK WORD,5.95,17.85
3,536368,22914,3,12/1/19 8:34,13047.0,United Kingdom,BLUE COAT RACK PARIS FASHION,4.95,14.85
4,536373,84406B,8,12/1/19 9:02,17850.0,United Kingdom,CREAM CUPID HEARTS COAT HANGER,2.75,22.00
...,...,...,...,...,...,...,...,...,...
54209,581585,84832,24,12/9/20 12:31,15804.0,United Kingdom,ZINC WILLIE WINKIE CANDLE STICK,0.85,20.40
54210,581585,23084,12,12/9/20 12:31,15804.0,United Kingdom,RABBIT NIGHT LIGHT,2.08,24.96
54211,581585,84692,25,12/9/20 12:31,15804.0,United Kingdom,BOX OF 24 COCKTAIL PARASOLS,0.42,10.50
54212,581587,22631,12,12/9/20 12:50,12680.0,France,CIRCUS PARADE LUNCH BOX,1.95,23.40


In [ ]:
#редактируем время:
df_prepared['InvoiceDate'] = pd.to_datetime(df_prepared['InvoiceDate'], unit = 's')
df_prepared

<ipython-input-98-3cf661a9c0e8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prepared['InvoiceDate'] = pd.to_datetime(df_prepared['InvoiceDate'], unit = 's')


,InvoiceNo,StockCode,Quantity,InvoiceDate,CustomerID,Country,Description,UnitPrice,Costs
0,536365,84029E,6,2019-12-01 08:26:00,17850.0,United Kingdom,RED WOOLLY HOTTIE WHITE HEART.,3.39,20.34
1,536367,84879,32,2019-12-01 08:34:00,13047.0,United Kingdom,ASSORTED COLOUR BIRD ORNAMENT,1.69,54.08
2,536367,21755,3,2019-12-01 08:34:00,13047.0,United Kingdom,LOVE BUILDING BLOCK WORD,5.95,17.85
3,536368,22914,3,2019-12-01 08:34:00,13047.0,United Kingdom,BLUE COAT RACK PARIS FASHION,4.95,14.85
4,536373,84406B,8,2019-12-01 09:02:00,17850.0,United Kingdom,CREAM CUPID HEARTS COAT HANGER,2.75,22.00
...,...,...,...,...,...,...,...,...,...
54209,581585,84832,24,2020-12-09 12:31:00,15804.0,United Kingdom,ZINC WILLIE WINKIE CANDLE STICK,0.85,20.40
54210,581585,23084,12,2020-12-09 12:31:00,15804.0,United Kingdom,RABBIT NIGHT LIGHT,2.08,24.96
54211,581585,84692,25,2020-12-09 12:31:00,15804.0,United Kingdom,BOX OF 24 COCKTAIL PARASOLS,0.42,10.50
54212,581587,22631,12,2020-12-09 12:50:00,12680.0,France,CIRCUS PARADE LUNCH BOX,1.95,23.40


In [ ]:
# время жизни пользователя:
df_life_time = df_prepared.groupby('CustomerID').agg({'InvoiceDate':['min','max'],'Costs':'count'})
lifetime_mean = df_life_time.loc[df_life_time['InvoiceDate']['max'] > df_life_time['InvoiceDate']['min']].mean()

<ipython-input-100-a8b62cfc0b69>:3: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  lifetime_mean = df_life_time.loc[df_life_time['InvoiceDate']['max'] > df_life_time['InvoiceDate']['min']].mean()


In [ ]:
df_life_time['lifetime'] = df_life_time['InvoiceDate']['max'] - df_life_time['InvoiceDate']['min']
df_life_time.sort_values('lifetime',ascending=False)



InvoiceDate                     Costs          lifetime
                           min                 max count                  
CustomerID                                                                
15311.0    2019-12-01 09:41:00 2020-12-09 12:00:00   255 374 days 02:19:00
12433.0    2019-12-01 13:24:00 2020-12-09 08:48:00    37 373 days 19:24:00
14606.0    2019-12-01 16:57:00 2020-12-08 19:28:00   278 373 days 02:31:00
14911.0    2019-12-01 14:05:00 2020-12-08 15:54:00   563 373 days 01:49:00
16210.0    2019-12-01 12:27:00 2020-12-08 12:36:00    12 373 days 00:09:00
...                        ...                 ...   ...               ...
13560.0    2020-12-02 13:11:00 2020-12-02 13:11:00     4   0 days 00:00:00
16457.0    2020-05-05 13:19:00 2020-05-05 13:19:00     1   0 days 00:00:00
13561.0    2020-05-08 12:39:00 2020-05-08 12:39:00     4   0 days 00:00:00
15264.0    2020-10-06 13:49:00 2020-10-06 13:49:00     3   0 days 00:00:00
16596.0    2019-12-08 16:18:00 2019-12-08 16:18:00     2   0 days 00:00:00

[3819 rows x 4 columns]

In [ ]:
# Сколько прибыли принес каждый пользователь
customer_profit = df_prepared.groupby('CustomerID').agg({'Costs':'sum'})
customer_profit.sort_values('Costs', ascending=False)

,Costs
CustomerID,
18102.0,26533.44
14646.0,25312.76
17450.0,18233.46
12415.0,16030.81
14911.0,14674.63
...,...
17603.0,-433.07
13564.0,-534.74
17867.0,-539.08


In [ ]:
df_incoming = df_prepared[df_prepared['Costs']>0].groupby('InvoiceNo').agg({'Costs':'sum'})
df_outcomes = df_prepared[df_prepared['Costs']<0].groupby('InvoiceNo').agg({'Costs':'sum'})
print(f"Выручка за период: {round(df_incoming['Costs'].sum(),2)}")
print(f"Расходы за период: {round(df_outcomes['Costs'].sum(),2)}")
print(f"Средний чек: {round(df_incoming['Costs'].sum()/len(df_incoming), 2)}")

Выручка за период: 1076979.04
Расходы за период: -84627.13
Средний чек: 76.97


In [ ]:
# Наиболее прибыльные товары:
df_profit_product = df_prepared[df_prepared['Costs']>0].groupby('Description').agg({'Costs':'sum','InvoiceNo':'count'})
df_profit_product.sort_values('Costs', ascending = False)

,Costs,InvoiceNo
Description,,
POSTAGE,21589.16,105
DOTCOM POSTAGE,20607.25,76
REGENCY CAKESTAND 3 TIER,17135.00,182
JUMBO BAG RED RETROSPOT,10092.03,219
WHITE HANGING HEART T-LIGHT HOLDER,9709.04,242
...,...,...
CHAMPAGNE TRAY BLANK CARD,0.57,1
SILICON STAR BULB BLUE,0.42,1
BLUE STONES ON WIRE FOR CANDLE,0.32,1


In [ ]:
# Наиболее продаваемый товар по количеству
df_profit_product.sort_values('InvoiceNo',ascending= False)

,Costs,InvoiceNo
Description,,
WHITE HANGING HEART T-LIGHT HOLDER,9709.04,242
JUMBO BAG RED RETROSPOT,10092.03,219
REGENCY CAKESTAND 3 TIER,17135.00,182
LUNCH BAG RED RETROSPOT,3795.92,167
PACK OF 72 RETROSPOT CAKE CASES,3452.93,161
...,...,...
SET 10 MINICARDS CUTE SNOWMAN 17071,1.66,1
LILY BROOCH OLIVE COLOUR,5.09,1
"STRING OF 8 BUTTERFLIES,PINK",4.95,1


In [ ]:
# наименее прибыльный товар
df_profit_product = df_prepared[df_prepared['Costs']<0].groupby('Description').agg({'Costs':'sum','InvoiceNo':'count'})
# цене:
df_profit_product.sort_values('Costs', ascending = True)

,Costs,InvoiceNo
Description,,
Manual,-15226.53,26
AMAZON FEE,-8847.55,2
"TUMBLER, NEW ENGLAND",-5253.36,1
TRAVEL CARD WALLET I LOVE LONDON,-4032.42,2
PANTRY CHOPPING BOARD,-3825.36,1
...,...,...
PLACE SETTING WHITE HEART,-0.42,1
MAGNETS PACK OF 4 HOME SWEET HOME,-0.39,1
GLASS CHALICE BLUE SMALL,-0.39,1


Предложения по дополнению датасета:
1. Количество товара на остатке. (Это поможет понять ликвидность каждого товара и принять решение что стоит докупать, от чего избавляться)

Рекомендации что ещё отслеживать:
1. Покупатели без ID/ Если мое предположение верно, что это часть из физического магазина, часть из интернет: для того чтобы учесть покупателей физического магазина, можно внедрить какие-то клиентские карты с программой лояльности.
2. У некоторых клиентов время жизни 0. Они раз купили и не вернулись. Можно предложить оценивать количество вернувшихся клиентов.
3. Продажи по месяцам/неделям, так сможем оценивать сезонные товары. Чем повысим прибыль в сезон.
4. Число посещающих сайт. Конверсию в платящих и тех кто не окончил покупку.
